In [2]:
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import networkx as nx
import requests
import json

import os

In [4]:
def get_colors(r):
  d = requests.get('http://api.scryfall.com/cards/named?fuzzy=' + str(r.replace(' ','+')))
  j = json.loads(d.text)

  if j['color_identity'] == []:
    return('None')
  else:
    return(str(j['color_identity']))
 
def get_deck_info(deck_url):
  page = requests.get(deck_url)
  soup = bs(page.content, 'lxml')  
  title = soup.find("td", attrs={'class':'S16'}).contents

  deck_name = re.split("#\d{1,2}(?:-\d)*",title[0])[1].split(" - ")[0].strip()
  player_name = title[1].text.strip()

  # # Get Prices
  prices = soup.find_all("span", attrs={'class':'O14'})
  price_breakout = [x.text for x in prices]
  return([deck_name,deck_url, player_name,int(price_breakout[0]),int(price_breakout[1]),float(price_breakout[2]),int(price_breakout[3])])  

# get_deck_info('http://www.mtgtop8.com/event?e=19361&d=323216&f=PAU')

get_colors('Delver of Secrets')

"['U']"

In [16]:
archetypes_URL = []
archetype_Names = []

# get Decks
formats = ['MO','ST',"PI",'PAU',"EDH"]

url = 'http://www.mtgtop8.com/format?f='+ formats[0]
page = requests.get(url)
soup = bs(page.content, 'html.parser')
for link in soup.find_all('a'):
    l = link.get('href')
    if 'archetype' in l:
        archetypes_URL.append('http://www.mtgtop8.com/' + link.get('href'))
        archetype_Names.append(link.text)
len(archetypes_URL)

61

In [17]:
deck_urls = []
for each in archetypes_URL:
    url = each
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')
    links = soup.find_all('a')
    for l in links:
        q = l.get('href')
        if '&d' in q:
            deck_urls.append('http://www.mtgtop8.com/' + q)
len(deck_urls)

468

In [19]:
import concurrent.futures 
deck_info = []
with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(get_deck_info, url): url for url in deck_urls}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
            deck_info.append(data)
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))

In [20]:
deck_info
df = pd.DataFrame(deck_info)
df.columns = ["Deck Name","URL","Player","Card Kingdom", "TCG Player", "Manatraders", "Cardhoarder"]
for c in df.columns:
  data_types = set(df[c].apply(type))
  print(c, data_types)
df

Deck Name {<class 'str'>}
URL {<class 'str'>}
Player {<class 'str'>}
Card Kingdom {<class 'int'>}
TCG Player {<class 'int'>}
Manatraders {<class 'float'>}
Cardhoarder {<class 'int'>}


Deck Name                                                URL  \
0        Red Deck Wins  http://www.mtgtop8.com/event?e=27330&d=416574&...   
1                 Jund  http://www.mtgtop8.com/event?e=27421&d=417722&...   
2                 Burn  http://www.mtgtop8.com/event?e=27296&d=416188&...   
3        Red Deck Wins  http://www.mtgtop8.com/event?e=27209&d=414098&...   
4                 Jund  http://www.mtgtop8.com/event?e=27257&d=414575&...   
..                 ...                                                ...   
463        Give Tokens  http://www.mtgtop8.com/event?e=27198&d=413998&...   
464  Jeskai Ascendancy  http://www.mtgtop8.com/event?e=27135&d=413446&...   
465         Living End  http://www.mtgtop8.com/event?e=27017&d=411719&...   
466   Allosaurus Combo  http://www.mtgtop8.com/event?e=26833&d=408548&...   
467        Give tokens  http://www.mtgtop8.com/event?e=26679&d=406569&...   

                Player  Card Kingdom  TCG Player  Manatraders  Cardhoarder  
0    pootisgodsamongus           173         597        34.95          342  
1           Joe Chaney          1366        1677        59.95          545  
2                 Boin           440         722        34.95          153  
3             The_nayr           185         613        34.95          353  
4            liuyuchen          1343        1724        59.95          554  
..                 ...           ...         ...          ...          ...  
463   Tanida Yoshiyuki           746         868        59.95          411  
464   Takahashi Masato           520        1492        34.95          203  
465       Nagata Akira           396         532        34.95          239  
466  Handou Shinnosuke           489         533        59.95          434  
467   Tanida Yoshiyuki           715         834        34.95          337  

[468 rows x 7 columns]

In [21]:
a = df.sort_values('Cardhoarder', ascending=True)

In [22]:
df[df['TCG Player']<100]

Empty DataFrame
Columns: [Deck Name, URL, Player, Card Kingdom, TCG Player, Manatraders, Cardhoarder]
Index: []

In [13]:
data = pd.DataFrame()
things = []
csv = "Deck,Player,Placement,Card \n"

for link in deck_urls:
    get_deck_info(link)


    player_name = p[1]
    placement = re.split('#',p[0])[1][0]
    cards = []
    colors = []
    for each in c:
        e = each.text
        cards.append(each.text)
#         print(e)
#         card_colors = get_colors(e)
#         colors.append(card_colors)
        
    for i in range(len(cards)):
#         csv = csv + deck + "," + player + "," + placement + "," + cards[i]+ "," + colors[i] + "\n"
           
      things.append((deck,player,placement,cards[i]))
            
# f= open("data.csv","w+")
# f.write(csv)
# f.close

NameError: name 'p' is not defined

In [ ]:
data = pd.DataFrame(things, columns=['Deck','Player','Placement','Card Name'])
data.head()

In [ ]:
  r = 'Delver of Secrets'
  d = requests.get('https://api.scryfall.com/cards/named?fuzzy=' + str(r.replace(' ','+')))
  j = json.loads(d.text)

  j
  
#   if j['colors'] == []:
#     return('None')
#   else:
#     return(str(j['colors']))


  